# Nomoto model first order PIT

In [ ]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

import copy

import numpy as np
import os

from src.data import database
from mdldb import mdl_to_evaluation
from mdldb.tables import Run
import src.data
import os.path

from sklearn.pipeline import Pipeline

import sympy as sp
from sklearn.metrics import r2_score
import src.reporting.paper_writing as paper_writing

from src.equations import equations
from src.equations import symbols

In [ ]:
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from pandas_profiling import ProfileReport
import evaluation.evaluation_helpers as evaluation_helpers

## Nomotos equation:

In [ ]:
Math(vlatex(equations.nomoto_first_order))

In [ ]:
db = database.get_db()

In [ ]:
sql="""
SELECT * from run
INNER JOIN projects
ON run.project_number==projects.project_number
    INNER JOIN loading_conditions
    ON (run.loading_condition_id == loading_conditions.id)
        INNER JOIN models
        ON run.model_number == models.model_number
            INNER JOIN ships
            ON models.ship_name == ships.name

WHERE run.test_type=="spiral"     
"""
data = pd.read_sql_query(sql=sql, con=db.engine)
data = data.loc[:,~data.columns.duplicated()]

In [ ]:
data.describe()

In [ ]:
#profile = ProfileReport(statistics, title='Pandas Profiling Report')
#profile.to_widgets()

In [ ]:
loading_conditions = data.groupby(by=['loading_condition_id','ship_speed'])

In [ ]:
loading_conditions.describe()

In [ ]:
#loading_condition = loading_conditions.get_group(name=(3,19))
loading_condition = loading_conditions.get_group(name=(144,16))

In [ ]:
loading_condition.describe()

### Load all data for one loading condition

In [ ]:
df_all = pd.DataFrame()
interesting_columns = ['delta','x0','y0','z0','phi','theta','psi']
for index, run in loading_condition.iterrows():
    db_run = db.session.query(Run).get(int(run.id))
    df = database.load_run(db_run=db_run)
    df['t'] = df.index 
    
    df_=evaluation_helpers.coord(df=df)  # add psi and position etc.
    
    df = pd.concat((df,df_), axis=1)

    df['run_id'] = run.id
    
    df_all = df_all.append(df[['t','run_id'] + interesting_columns], ignore_index=True)

In [ ]:
df_all.describe()

In [ ]:
fig,ax=plt.subplots()
runs = df_all.groupby(by='run_id')
for run_id, df in runs:
    df['x0']-=df.iloc[0]['x0']
    df['y0']-=df.iloc[0]['y0']
    
    df.plot(x='y0',y='x0', ax=ax)
    
ax.get_legend().remove()
ax.set_aspect('equal', 'box')

In [ ]:
def derivate(group):
    df = group.set_index('t')
    
    ddf = np.gradient(df, df.index, axis=0).mean(axis=0)
    s = pd.Series(ddf, index=df.columns)
    
    return s

In [ ]:
df = runs.mean()

ddf = runs.apply(func= derivate)

df['u']=ddf['x0']
df['v']=ddf['y0']
df['w']=ddf['z0']
df['p']=ddf['phi']
df['q']=ddf['theta']
df['r']=ddf['psi']


In [ ]:
fig,ax=plt.subplots()
df.plot(x='delta', y='r', ax=ax, style='o')
ax.grid(True)
ax.set_title('Reverse spiral plot')

In [ ]:
print(loading_condition.iloc[0]['project_path'])